In [ ]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import auc

In [ ]:
CVX_FOLDER = 'results_regression/cvx_regression_baseline/'
OUR_FOLDER = 'results/NN_energy_regression/'

In [ ]:
our_results = [file for file in os.listdir(OUR_FOLDER) if not file[0]=='.']
cvx_results = [file for file in os.listdir(CVX_FOLDER) if not file[0]=='.']

In [ ]:
datasets_ours = set([file.split('_')[0] for file in our_results])
datasets_cvx = set([file.split('_')[0] for file in cvx_results])

In [ ]:
fair_metric = 'statistical_parity'
acc_metric = 'R2'

In [ ]:
our_method = 'linearregression'
dataset = 'StudentsPortugese'
dset = 'test'

aucs = []
for i in range(10):
    res = pd.read_csv('{}{}_{}_{}_{}.csv'.format(OUR_FOLDER, dataset, our_method, dset, i), index_col=0)[[fair_metric, acc_metric]]
    res[res<0.0] = 0
    res = res.append({
        fair_metric: 1,
        acc_metric : res[acc_metric].max()
    }, ignore_index=True)
    res = res.append({
        fair_metric: res[fair_metric].min(),
        acc_metric : 0
    }, ignore_index=True).sort_values(fair_metric)
    aucs.append(auc(res[fair_metric], res[acc_metric]))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import colors as mcolors
from matplotlib import rcParams
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
s_ = 140

In [ ]:
def plot_dataset(dataset, dset, ylim, plot_legend=True, AUC=False, errorbars=False, title=False):
    print(f'{dataset}')
    if dset == 'train':
        print('Training times')
    else:
        print('AUCs')
    for our_method in ['NN', 'linearregression']:
        our_results = [pd.read_csv('{}{}_{}_{}_{}.csv'.format(OUR_FOLDER, dataset, our_method, dset, i), index_col=0) for i in range(10)]
        our_results = pd.concat(our_results).groupby('lambda_').agg(['mean', 'std'])
        # compute AUC`
        aucs = []
        times = []
        for i in range(10):
            if dset == 'train':
                times.append(pd.read_csv('{}{}_{}_{}_{}.csv'.format(OUR_FOLDER, dataset, our_method, dset, i), index_col=0).time)
            res = pd.read_csv('{}{}_{}_{}_{}.csv'.format(OUR_FOLDER, dataset, our_method, dset, i), index_col=0)[[fair_metric, acc_metric]]
            res[res<0.0] = 0
            res = res.append({
                fair_metric: 1,
                acc_metric : res[acc_metric].max()
            }, ignore_index=True)
            res = res.append({
                fair_metric: res[fair_metric].min(),
                acc_metric : 0
            }, ignore_index=True).sort_values(fair_metric)
            aucs.append(auc(res[fair_metric], res[acc_metric]))
        if errorbars:
            plt.errorbar(our_results[fair_metric]['mean'], 
                         our_results[acc_metric]['mean'],
                         xerr = our_results[fair_metric]['std'], 
                         yerr = our_results[acc_metric]['std'], 
                         fmt='o', 
                         ecolor='lightgrey',
                         label='MFL ({}): AUC={}({})'.format('NN' if our_method=='NN' else 'Linear', np.mean(aucs).round(3), np.std(aucs).round(3)) if AUC 
                         else 'MFL ({})'.format('NN' if our_method=='NN' else 'Linear'))
        else:
            plt.scatter(our_results[fair_metric]['mean'], 
                         our_results[acc_metric]['mean'],
                         marker = 'o' if our_method=='NN' else 'v',
                         label='MFL ({}): AUC={}({})'.format('NN' if our_method=='NN' else 'Linear', np.mean(aucs).round(3), np.std(aucs).round(3)) if AUC 
                         else 'MFL ({})'.format('NN' if our_method=='NN' else 'Linear'),
                         s=s_, edgecolors='k', alpha=.7,
                         c=colors['limegreen'] if our_method=='NN' else colors['yellow'])
        if dset == 'train':
            print(f'Ours {our_method}: {np.abs(times).mean()}')
        else:
            print(f'Ours {our_method}: {np.mean(aucs)}({np.std(aucs)})')
            

    for cvx_method in ['individual', 'group']:
        cvx_results = [pd.read_csv('{}{}_cvx-bl-0_{}_{}_{}.csv'.format(CVX_FOLDER, dataset, cvx_method, dset, i), index_col=0) for i in range(10)]
        cvx_results = pd.concat(cvx_results).groupby('lambda_').agg(['mean', 'std'])
        # compute AUC
        aucs = []
        times = []
        for i in range(10):
            if dset == 'train':
                times.append(pd.read_csv('{}{}_cvx-bl-0_{}_{}_{}.csv'.format(CVX_FOLDER, dataset, cvx_method, dset, i), index_col=0).time)
            res = pd.read_csv('{}{}_cvx-bl-0_{}_{}_{}.csv'.format(CVX_FOLDER, dataset, cvx_method, dset, i), index_col=0)[[fair_metric, acc_metric]]
            res[res<0.0] = 0
            res = res.append({
                fair_metric: 1,
                acc_metric : res[acc_metric].max()
            }, ignore_index=True)
            res = res.append({
                fair_metric: res[fair_metric].min(),
                acc_metric : 0
            }, ignore_index=True).sort_values(fair_metric)
            aucs.append(auc(res[fair_metric], res[acc_metric]))
        
        if errorbars:
            plt.errorbar(cvx_results[fair_metric]['mean'], 
                         cvx_results[acc_metric]['mean'],
                         xerr = cvx_results[fair_metric]['std'], 
                         yerr = cvx_results[acc_metric]['std'], 
                         fmt='o', 
                         ecolor='lightgrey',
                         label='Berk et al. ({}): AUC={}({})'.format(cvx_method, np.mean(aucs).round(3), np.std(aucs).round(3)) if AUC
                         else 'Berk et al. ({})'.format(cvx_method))
        else:
            plt.scatter(cvx_results[fair_metric]['mean'], 
                         cvx_results[acc_metric]['mean'],
                         marker = 's' if cvx_method=='individual' else 'P',
                         label='Berk et al. ({}): AUC={}({})'.format(cvx_method, np.mean(aucs).round(3), np.std(aucs).round(3)) if AUC
                         else 'Berk et al. ({})'.format(cvx_method),
                         s=s_, edgecolors='k', alpha=.7,
                         c=colors['indianred'] if cvx_method=='individual' else colors['darkblue'])
        if dset == 'train':
            print(f'CVX {cvx_method}: {np.abs(times).mean()}')
        else:
            print(f'CVX  {cvx_method}: {np.mean(aucs)}({np.std(aucs)})')
    #plt.ylim(ylim)
    plt.xlabel('SP (Kolmogorov distance)')
    plt.ylabel(acc_metric)
    if title:
        plt.title('{} Results'.format(dset.capitalize()))
        
    if plot_legend:
        plt.legend(loc='lower right')
    rcParams['font.family'] = 'serif'
    rcParams['font.sans-serif'] = ['Times']
    rcParams.update({'font.size': 15})
    plt.grid(alpha=.3)
    plt.tight_layout()

In [ ]:
ylim = {
    'CommunitiesCrime' : (0,0.2),
    'StudentsMath' : (0,30),
    'StudentsPortugese' : (0,30)
} if acc_metric == 'MSE' else {
    'CommunitiesCrime' : (-0.2,1),
    'StudentsMath' : (-0.2,1),
    'StudentsPortugese' : (-0.2,1)
}

In [ ]:
for dataset in datasets_cvx:
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plot_dataset(dataset, 'train', ylim[dataset])
    plt.subplot(1,2,2)
    plot_dataset(dataset, 'test', ylim[dataset])
    plt.suptitle(dataset)
    plt.show()

In [ ]:
for dataset in datasets_cvx:
    plot_dataset(dataset, 'test', ylim[dataset])
    plt.ylim(0,1)
    plt.savefig(f'plots/regression_{dataset}.pdf')
    plt.show()